<img src="https://drive.google.com/uc?export=view&id=1WNLKH10YpQNNk9eeRIyYLwGkxNbNp-Mm" width="100%">

# Modelamiento y Evaluación
---

Este notebook es una plantilla que le puede servir como guía para el cuarto entregable del proyecto aplicado.

## **1. Tipo de Modelamiento**
---

Seleccione el tipo de modelo que puede usar para dar respuesta al problema de NLP. Recuerde que dispone de modelos que incluyen:

- **Regresión**: se busca estimar un valor continúo a partir de los textos.
- **Clasificación**: permite estimar un valor categórico a partir de los textos.
- **Agrupamiento**: permite encontrar grupos de documentos similares.
- **Modelos de tópicos**: permite encontrar temas/tópicos dentro del corpus e interpretarlo.
- **Otros modelos**: recuerde que dispone de otros tipos de tareas supervisadas y no supervisadas desde NLP. Puede utilizar modelos pre-enternados con el fin de dar una solución.

Justifique la selección del modelo.

---**INGRESE SU RESPUESTA**---

## **2. Implementación del modelo**
---

A continuación encuentra los pasos que puede seguir para la implementación de un modelo:

### **2.1. Importar Modelo**
---

Seleccione la librería de NLP que utilizará para el modelo e impórtelo:

In [ ]:
import sklearn
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV

# modelos de ML
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor

### **2.2. Entrenamiento del Modelo y Selección de Hiperparámetros**
---

Si está utilizando un modelo que requiere entrenamiento, deberá entrenarlo y seleccionar un conjunto de hiperparámetros válido para el mismo:

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    vects_arr, comments["toxicity"].values, test_size=0.2, random_state=42
)

In [ ]:
param_dist = {
    "n_estimators": np.random.randint(10, 500, 10).tolist(),
    "max_depth": np.random.randint(2, 20, 10).tolist(),
    "min_samples_split": np.random.randint(2, 20, 10).tolist(),
    "min_samples_leaf": np.random.randint(1, 20, 10).tolist(),
}

# Create the Random Forest Regression model
rf_regressor = RandomForestRegressor(random_state=42)

# Create the random search object
grid_search_rf = RandomizedSearchCV(
    estimator=rf_regressor,
    param_distributions=param_dist,
    n_iter=3,
    cv=3,
    scoring="neg_median_absolute_error",
    random_state=42,
    n_jobs=-1,
)

grid_search_rf.fit(X_train, y_train)
best_model_rf = grid_search_rf.best_estimator_

Best Parameters:  {'n_estimators': 201, 'min_samples_split': 16, 'min_samples_leaf': 1, 'max_depth': 8}
Mean Squared Error:  0.035433787247476776


In [ ]:

model = MLPRegressor()

# Define the hyperparameters to tune
param_grid = {
    "hidden_layer_sizes": [(50,), (10, 5), (20, 15)],
    "activation": ["relu", "tanh"],
    "alpha": [0.001, 0.01, 0.1, 0.5],
    "learning_rate": ["constant", "adaptive"],
    "max_iter": [100, 200, 500],
}

# Create a grid search object
grid_search_nn = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    scoring="neg_mean_squared_error",
    cv=4,
    n_jobs=-1,
)
grid_search_nn = grid_search_nn.fit(X_train, y_train)
best_model_nn = grid_search_nn.best_estimator_

## **3. Evaluación o Aplicación del modelo**
---

Si entrenó un modelo, recuerde que debe reportar el desempeño del mismo sobre un conjunto de datos no visto (test). Considere que dispone de las siguientes métricas:

- **Clasificación**: accuracy, precision, recall, f1-score, AUC.
- **Regresión**: $r^2$, error cuadrático medio, error absoluto medio.
- **Agrupamiento**: coeficiente de silueta, índice de Davies-Bouldin.
- **Tópicos**: perplexity, score de coherencia.

In [ ]:
best_model = grid_search_rf.best_estimator_

# metricas de desempeño
y_pred = best_model.predict(X_train)
rmse = np.sqrt(sklearn.metrics.mean_squared_error(y_train, y_pred))
mse = sklearn.metrics.mean_squared_error(y_train, y_pred)
r2 = sklearn.metrics.r2_score(y_train, y_pred)
print("Best Parameters: ", grid_search_rf.best_params_)
print(f" RMSE: {rmse:.4f}\n MSE: {mse:.4f}\n R2: {r2:.4f}")

# grafica de predicciones vs valores reales
predicciones = (
    pd.DataFrame({"y_true": y_train, "y_pred": y_pred})
    .reset_index()
    .sort_values(by="y_true")
)
fig = px.scatter(predicciones, x=predicciones.index, y="y_pred", opacity=0.5)
fig.add_scatter(
    x=predicciones.index, y=predicciones["y_true"], mode="markers", opacity=0.5
)
fig.show()

Best Parameters:  {'n_estimators': 201, 'min_samples_split': 16, 'min_samples_leaf': 1, 'max_depth': 8}
 RMSE: 0.1800
 MSE: 0.0324
 R2: 0.1092


In [ ]:
best_model = grid_search_nn.best_estimator_

# metricas de desempeño
y_pred = best_model.predict(X_test)
rmse = np.sqrt(sklearn.metrics.mean_squared_error(y_test, y_pred))
mse = sklearn.metrics.mean_squared_error(y_test, y_pred)
r2 = sklearn.metrics.r2_score(y_test, y_pred)
print("Best Parameters: ", grid_search_nn.best_params_)
print(f" RMSE: {rmse:.4f}\n MSE: {mse:.4f}\n R2: {r2:.4f}")

# grafica de predicciones vs valores reales
predicciones = (
    pd.DataFrame({"y_true": y_test, "y_pred": y_pred})
    .reset_index()
    .sort_values(by="y_true")
)
fig = px.scatter(predicciones, x=predicciones.index, y="y_pred", opacity=0.5)
fig.add_scatter(
    x=predicciones.index, y=predicciones["y_true"], mode="markers", opacity=0.5
)
fig.show()

Best Parameters:  {'activation': 'relu', 'alpha': 0.01, 'hidden_layer_sizes': (10, 5), 'learning_rate': 'adaptive', 'max_iter': 200}
 RMSE: 0.1866
 MSE: 0.0348
 R2: -0.0006


## **Créditos**

* **Profesor:** [Felipe Restrepo Calle](https://dis.unal.edu.co/~ferestrepoca/)
* **Asistentes docentes:**
    - [Juan Sebastián Lara Ramírez](https://www.linkedin.com/in/juan-sebastian-lara-ramirez-43570a214/).
* **Diseño de imágenes:**
    - [Rosa Alejandra Superlano Esquibel](mailto:rsuperlano@unal.edu.co).
* **Coordinador de virtualización:**
    - [Edder Hernández Forero](https://www.linkedin.com/in/edder-hernandez-forero-28aa8b207/).

**Universidad Nacional de Colombia** - *Facultad de Ingeniería*